In [8]:
fs = 1; Δt = 100
using Immerse
using Gtk.ShortNames,ColorTypes
using GtkUtilities:copy!
type State
    name::Symbol
    price::Float64
end

win = @Window() |>  (g = @Grid())

play_ = @Button("play");   g[1:2,1] = play_
pause_ = @Button("pause"); g[3:4,1] = pause_
exit_ = @Button("exit");   g[5:6,1] = exit_
buy_ = @Button("buy");     g[1:3,2] = buy_
sell_ = @Button("sell");   g[4:6,2] = sell_
c = @Canvas();             g[1:6,3:10] = c

setproperty!(c, :expand, true)
showall(win)

signal_connect(buy_, "clicked") do widget
    @eval if state.name == :sell
        adv_price = sell_price[t-Δt]  # adversary's price
        profit += state.price - 0.2 - (adv_price + 0.2)
        state.name = :empty
        state.price = adv_price
    elseif state.name == :empty
        adv_price = sell_price[t-Δt]  # adversary's price
        state.name = :buy
        state.price = adv_price
    end
end
signal_connect(sell_, "clicked") do widget
    @eval if state.name == :buy
        adv_price = buy_price[t-Δt]   
        profit += adv_price - 0.2 - (state.price + 0.2)
        state.name = :empty
        state.price = adv_price
    elseif state.name == :empty
        adv_price = buy_price[t-Δt]
        state.name = :sell
        state.price = adv_price
    end
end
signal_connect(play_, "clicked") do widget
    @eval flag = true
end
signal_connect(pause_, "clicked") do widget
    @eval flag = false
end
signal_connect(exit_, "clicked") do widget
    @eval close(timer)
end


# file = open_dialog("Pick an csv file")
file = "IF1307.csv"
data = readcsv(file)
data[data.==""] = 0.0
buy_price, sell_price = data[10:end-10,18],data[10:end-10,20]
buy_price = convert(Array{Float64},buy_price)
sell_price = convert(Array{Float64},sell_price);
t = 10
flag = false;
profit = 0
state = State(:empty,0)
theme1 = Theme(default_color=colorant"blue")
theme2 = Theme(default_color=colorant"green")
range = Coord.cartesian(xmin=1, xmax=length(buy_price),
ymin=minimum(buy_price), ymax=maximum(buy_price))
timer = Timer(0,1/fs) do timer
    global t
    if t <= length(buy_price) && flag
        c.back = Gtk.cairo_surface_for(c)
        x = max(t-5000+1,1):t
        p = plot(layer(x=x,y=buy_price[x], Geom.line, theme1),
#             layer(x=x,y=sell_price[x], Geom.line, theme2),
        Guide.title(@sprintf("profit=%4.2f state=%s",profit,state.name)),
        layer(yintercept=[state.price], Geom.hline),
        range)
        display(c,Figure(p))
        t += Δt
    elseif t > length(buy_price)
        close(timer)
    end
end


Timer(Ptr{Void} @0x000000000bb82570,Condition(Any[Task (waiting) @0x00007f4072f7c970]),true)